In [299]:
# Montar/treinar o modelo e verificar a precisão

In [3]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from category_encoders import OneHotEncoder
from datetime import datetime


2025-06-20 22:16:47.241038: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 22:16:47.241479: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-20 22:16:47.243976: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-20 22:16:47.250235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750468607.260022  181769 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750468607.26

In [349]:
df = pd.read_csv("./data-fict/Encoded_Data_Ocorr.csv")
df.head()

,data_ocorrencia,hora_ocorrencia,regiao_administrativa,tipo_crime,sexo_vitima,idade_vitima,risco
0,2023-01-14,21:00,0,0,2,19,5.241323e-08
1,2023-01-20,02:00,0,0,2,17,5.241323e-08
2,2023-01-24,20:15,0,0,2,30,5.241323e-08
3,2023-01-03,09:30,0,2,1,41,5.241323e-08
4,2023-01-18,20:00,0,2,1,54,5.241323e-08


In [350]:
## Separação da variável dependente

X = df.drop(columns=["risco", "tipo_crime"])
y = df["risco"]

In [351]:
y.head()

0    5.241323e-08
1    5.241323e-08
2    5.241323e-08
3    5.241323e-08
4    5.241323e-08
Name: risco, dtype: float64

In [370]:
# Imputação dos dados
## Isso aqui transforma os valores entre 0 e 1 

scaler = MinMaxScaler()
Y = scaler.fit_transform(y.values.reshape(-1, 1))
Y = scaler.transform(y.values.reshape(-1, 1))

Y = pd.DataFrame(Y, columns=["risco"])
Y.to_csv("./a.csv", index=True)

In [334]:
print(Y)

[[0.00132747]
 [0.00132747]
 [0.00132747]
 ...
 [0.0111592 ]
 [0.0111592 ]
 [0.0111592 ]]


In [371]:
## Isso aqui transforma os valores entre 0 e 1 
scaler = MinMaxScaler()
Y = scaler.fit_transform(y.values.reshape(-1, 1))
Y = scaler.transform(y.values.reshape(-1, 1))

def classifica_valor(x):
    if x > 0.8: 
        return 5
    elif x > 0.6:
        return 4
    elif x > 0.4:
        return 3
    elif x > 0.2:
        return 2
    elif x >= 0.0:
        return 1

Y = pd.DataFrame(Y, columns=["risco"])
Y['risco'] = Y['risco'].apply(classifica_valor)
Y.to_csv("./b.csv", index=True)

In [60]:
test = pd.read_csv("./data-fict/Ocorrencias_Com_Risco.csv")

test_teste = test.drop(columns=["risco", "tipo_crime"])
X = test_teste

test_risco = test["risco"]
Y = test_risco


In [61]:
# Conversão da hora string para minutos

def tempo_para_minutos(tempo):
    # Divide a string 'hh:mm' em horas e minutos
    horas, minutos = map(int, tempo.split(':'))
    
    # Converte tudo para minutos
    total_minutos = horas * 60 + minutos
    return total_minutos

X["hora_ocorrencia"] = X["hora_ocorrencia"].apply(tempo_para_minutos)

X.head()

,data_ocorrencia,hora_ocorrencia,regiao_administrativa,sexo_vitima,idade_vitima
0,2023-01-14,1260,0,2,19
1,2023-01-20,120,0,2,17
2,2023-01-24,1215,0,2,30
3,2023-01-03,570,0,1,41
4,2023-01-18,1200,0,1,54


In [62]:
def data_para_dias(data_str):
    data = datetime.strptime(data_str, r'%Y-%m-%d')
    referencia = datetime(1970, 1, 1)
    return (data - referencia).days

X["data_ocorrencia"] = X["data_ocorrencia"].apply(data_para_dias)

X.head()

,data_ocorrencia,hora_ocorrencia,regiao_administrativa,sexo_vitima,idade_vitima
0,19371,1260,0,2,19
1,19377,120,0,2,17
2,19381,1215,0,2,30
3,19360,570,0,1,41
4,19375,1200,0,1,54


In [90]:
# Isso aqui divide o treino do teste com uma proporção de 0.5 
## Se esse valor ficar muito grande, a eficiência pode ser reduzida porque o modelo começa a decorar os dados

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

In [91]:
print(X_train)

      data_ocorrencia  hora_ocorrencia  regiao_administrativa  sexo_vitima  \
4750            19385              510                      9            1   
9884            19367             1110                     29            2   
6010            19358              540                     17            2   
5676            19364              600                     16            2   
7249            19378              390                     20            2   
...               ...              ...                    ...          ...   
5734            19367              930                     16            1   
5191            19381             1080                     14            2   
5390            19376              855                     16            2   
860             19375              270                      1            1   
7270            19388              240                     20            1   

      idade_vitima  
4750            37  
9884            50  


In [92]:
# Imputação dos dados
## Isso aqui transforma os valores entre 0 e 1 

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [93]:
print(X_train_scaled)

[[0.9        0.35789474 0.28125    0.         0.43103448]
 [0.3        0.77894737 0.90625    1.         0.65517241]
 [0.         0.37894737 0.53125    1.         0.56896552]
 ...
 [0.6        0.6        0.5        1.         0.10344828]
 [0.56666667 0.18947368 0.03125    0.         0.37931034]
 [1.         0.16842105 0.625      0.         0.5       ]]


In [ ]:
# tf.random.set_seed(2)
#

n_classes = y_train.nunique()

# Construção das camadas
inp = keras.layers.Input((X_train_scaled.shape[1], ))
hid = keras.layers.Dense(4, activation="relu")(inp)

out_soft = keras.layers.Dense(n_classes, activation="softmax")(hid)
# out_sig = keras.layers.Dense(units=1, activation="sigmoid")(drop)


# model_sig = keras.Model(inp, out_sig)
model_soft = keras.Model(inp, out_soft)

# model_sig.compile(optimizer='adam',
#               loss='msle',
#               metrics=['accuracy'])
model_soft.compile(optimizer='adam',
                   loss="sparse_categorical_crossentropy",
                   metrics=["accuracy"])

# model_sig.summary()
model_soft.summary()

IndexError: tuple index out of range

In [95]:
es = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10, mode="min", restore_best_weights=True,
)

In [105]:
print(np.array(X_train_scaled).shape)
print(np.array(X_train_scaled))

(5218, 5)
[[0.9        0.35789474 0.28125    0.         0.43103448]
 [0.3        0.77894737 0.90625    1.         0.65517241]
 [0.         0.37894737 0.53125    1.         0.56896552]
 ...
 [0.6        0.6        0.5        1.         0.10344828]
 [0.56666667 0.18947368 0.03125    0.         0.37931034]
 [1.         0.16842105 0.625      0.         0.5       ]]


In [102]:
model_soft.fit(x=X_train_scaled, y=y_train, validation_data=(X_test_scaled, y_test), shuffle=True, batch_size=32, callbacks=[es], verbose=1, epochs=20)

Epoch 1/20


2025-06-20 22:51:45.658526: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 5 which is outside the valid range of [0, 5).  Label values: 3 3 3 5 3 2 3 4 1 5 2 2 5 3 1 4 3 2 4 5 4 4 2 4 5 5 2 4 4 4 4 2


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/garciavp/-ruelers.h-AI/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/garciavp/-ruelers.h-AI/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/garciavp/-ruelers.h-AI/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/garciavp/-ruelers.h-AI/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.1

In [88]:
# Testar modelo

loss, accuracy = model_soft.evaluate(X_test_scaled, y_test, batch_size=32)
print(f'Loss no conjunto de teste: {loss:.4f}')
print(f'Accuracy no conjunto de teste: {accuracy:.4f}')

164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3684 - loss: 1.4199
Loss no conjunto de teste: 1.4172
Accuracy no conjunto de teste: 0.3752


In [89]:
model_soft.save("./modelos/modelo_crime.keras")